## Residual Network

In [1]:
!nvidia-smi

Sat Mar 19 19:36:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:0B.0 Off |                    0 |
| N/A   36C    P0    54W / 300W |   1059MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import paddle
import numpy as np

paddle.__version__, paddle.device.get_device()

('2.2.2', 'gpu:0')

### Data loading

In [3]:
import paddle.vision.transforms as T
from paddle.vision.datasets import FashionMNIST

# loading and normalization
# resize the shape to 96x96
transform = [T.Resize(size=96), T.Normalize(mean=[127.5], std=[127.5])]
transform = T.Compose(transform)

# constructing traning set and test set
fashionmnist_train = FashionMNIST(mode='train', transform=transform)
fashionmnist_test = FashionMNIST(mode='test', transform=transform)

In [4]:
# nums of train set and test set
len(fashionmnist_train), len(fashionmnist_test)

(60000, 10000)

In [5]:
# As for one sample, channel = 1, height, weight = 96
# for adapting to the input shape of resnet
fashionmnist_train[0][0].shape, fashionmnist_test[0][0].shape

((1, 96, 96), (1, 96, 96))

In [6]:
train_loader = paddle.io.DataLoader(fashionmnist_train, batch_size=256, shuffle=True)
test_loader = paddle.io.DataLoader(fashionmnist_test, batch_size=64, shuffle=False)

### Model


#### Residual block

In [8]:
from paddle import nn
from paddle.nn import functional as F

class Residual(nn.Layer): 
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2D(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2D(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2D(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2D(num_channels)
        self.bn2 = nn.BatchNorm2D(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [12]:
# where the input and output are of the same shape
blk = Residual(3, 3)
X = paddle.rand([4, 3, 6, 6])
Y = blk(X)
Y.shape

[4, 3, 6, 6]

In [13]:
# halve the output height and width while increasing the number of output channels
blk = Residual(3, 6, use_1x1conv=True, strides=2)
blk(X).shape 

[4, 6, 3, 3]

#### Residual Network

In [18]:
b1 = nn.Sequential(nn.Conv2D(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2D(64), nn.ReLU(),
                   nn.MaxPool2D(kernel_size=3, stride=2, padding=1))

In [19]:
# special processing has been performed on the first module
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [20]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [22]:
resnet = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2D((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))

In [23]:
# instantiation
model = paddle.Model(resnet)

# visualization of the model workflow
model.summary((-1, 1, 96, 96)) # [N C H W]

-------------------------------------------------------------------------------
   Layer (type)         Input Shape          Output Shape         Param #    
     Conv2D-31        [[1, 1, 96, 96]]     [1, 64, 48, 48]         3,200     
  BatchNorm2D-27     [[1, 64, 48, 48]]     [1, 64, 48, 48]          256      
      ReLU-1         [[1, 64, 48, 48]]     [1, 64, 48, 48]           0       
    MaxPool2D-1      [[1, 64, 48, 48]]     [1, 64, 24, 24]           0       
     Conv2D-32       [[1, 64, 24, 24]]     [1, 64, 24, 24]        36,928     
  BatchNorm2D-28     [[1, 64, 24, 24]]     [1, 64, 24, 24]          256      
     Conv2D-33       [[1, 64, 24, 24]]     [1, 64, 24, 24]        36,928     
  BatchNorm2D-29     [[1, 64, 24, 24]]     [1, 64, 24, 24]          256      
    Residual-14      [[1, 64, 24, 24]]     [1, 64, 24, 24]           0       
     Conv2D-34       [[1, 64, 24, 24]]     [1, 64, 24, 24]        36,928     
  BatchNorm2D-30     [[1, 64, 24, 24]]     [1, 64, 24, 24]    

{'total_params': 11186186, 'trainable_params': 11170570}

### Training

In [24]:
# optimizer and loss
model.prepare(optimizer=paddle.optimizer.Adam(parameters=model.parameters(), learning_rate=1e-3),
              loss=nn.CrossEntropyLoss(),
              metrics=paddle.metric.Accuracy())

# training
model.fit(train_loader,
        # eval_data = test_loader
        epochs=20,
        verbose=1,
        )

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/20


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:653: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


step 235/235 [==============================] - loss: 0.3162 - acc: 0.8552 - 77ms/step          
Epoch 2/20
step 235/235 [==============================] - loss: 0.2791 - acc: 0.9125 - 77ms/step          
Epoch 3/20
step 235/235 [==============================] - loss: 0.1962 - acc: 0.9288 - 78ms/step          
Epoch 4/20
step 235/235 [==============================] - loss: 0.1297 - acc: 0.9445 - 77ms/step          
Epoch 5/20
step 235/235 [==============================] - loss: 0.2486 - acc: 0.9522 - 81ms/step          
Epoch 6/20
step 235/235 [==============================] - loss: 0.1365 - acc: 0.9610 - 81ms/step          
Epoch 7/20
step 235/235 [==============================] - loss: 0.1179 - acc: 0.9692 - 82ms/step          
Epoch 8/20
step 235/235 [==============================] - loss: 0.0710 - acc: 0.9739 - 83ms/step          
Epoch 9/20
step 235/235 [==============================] - loss: 0.0805 - acc: 0.9781 - 80ms/step          
Epoch 10/20
step 235/235 [=============

In [25]:
model.evaluate(fashionmnist_test, batch_size=64, verbose=1)

Eval begin...
step 157/157 [==============================] - loss: 0.0078 - acc: 0.9227 - 16ms/step          
Eval samples: 10000


{'loss': [0.007836733], 'acc': 0.9227}